In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import ast
import math
import nltk

In [8]:
credits=pd.read_csv("../dataset/credits.csv")
movies=pd.read_csv("../dataset/movies_metadata.csv")
ratings=pd.read_csv("../dataset/ratings.csv")


C:\Users\USER\AppData\Local\Temp\ipykernel_14680\3440597929.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies=pd.read_csv("../movie/movies_metadata.csv")


In [9]:
credits["id"]=credits["id"].astype("str")
moviesall=movies.merge(credits,on="id")
moviesall=moviesall.fillna(" ")
moviesall.to_csv("moviesall.csv",index=False)


In [10]:
moviesall=moviesall.drop('spoken_languages',axis=1)
moviesall=moviesall.drop('production_countries',axis=1)
moviesall=moviesall.drop('budget',axis=1)
moviesall["title1"]=moviesall["title"]



In [11]:
def preprocessing(obj):
    a=[]
    count=0
    if obj!=" ":
        for i in ast.literal_eval(obj):
            if count!=3:
                a.append(i["name"])

                count+=1
            else:
                break
        return a 
    else:
        return obj
def preprocessing1(obj):
    
    a=[]
    count=0
    for i in ast.literal_eval(obj):
        if count!=3:
            a.append(i["name"])
            a.append(i["character"])
            count+=1
        else:
            break
    return a 
def preprocessing2(obj):
    for i in ast.literal_eval(obj):
        if i["job"]=="Director":
            return [i['name']]
    return [""]
        
def preprocessing3(obj):
    if obj!=' ':
         return ast.literal_eval(obj)["name"]
    else:
        return obj


    
    
   

In [12]:
moviesall["crew"]=moviesall["crew"].apply(preprocessing2)
moviesall["belongs_to_collection"]=moviesall["belongs_to_collection"].apply(preprocessing3)
moviesall["genres"]=moviesall["genres"].apply(preprocessing)
moviesall["cast"]=moviesall["cast"].apply(preprocessing1)
moviesnew=moviesall[["belongs_to_collection","genres","id","imdb_id","overview","popularity","production_companies","title","tagline","vote_average","vote_count","crew","cast","adult","title1"]]
moviesnew["overview"]=moviesnew["overview"].apply(lambda x:x.split())


C:\Users\USER\AppData\Local\Temp\ipykernel_14680\1900880400.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moviesnew["overview"]=moviesnew["overview"].apply(lambda x:x.split())


In [13]:
moviesnew["title"]=moviesnew["title"].apply(lambda x:x.split())
moviesnew["tagline"]=moviesnew["tagline"].apply(lambda x:x.split())
moviesnew["belongs_to_collection"]=moviesnew["belongs_to_collection"].apply(lambda x:x.split())

C:\Users\USER\AppData\Local\Temp\ipykernel_14680\4194663555.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moviesnew["title"]=moviesnew["title"].apply(lambda x:x.split())
C:\Users\USER\AppData\Local\Temp\ipykernel_14680\4194663555.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moviesnew["tagline"]=moviesnew["tagline"].apply(lambda x:x.split())
C:\Users\USER\AppData\Local\Temp\ipykernel_14680\4194663555.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [14]:
moviesnew["belongs_to_collection"]=moviesnew["belongs_to_collection"].apply(lambda x:[i.replace(" ","") for i in x])
moviesnew["genres"]=moviesnew["genres"].apply(lambda x:[i.replace(" ","") for i in x])
moviesnew["overview"]=moviesnew["overview"].apply(lambda x:[i.replace(" ","") for i in x])
moviesnew["title"]=moviesnew["title"].apply(lambda x:[i.replace(" ","") for i in x])
moviesnew["tagline"]=moviesnew["tagline"].apply(lambda x:[i.replace(" ","") for i in x])
moviesnew["crew"]=moviesnew["crew"].apply(lambda x:[i.replace(" ","") for i in x])
moviesnew["cast"]=moviesnew["cast"].apply(lambda x:[i.replace(" ","") for i in x])

C:\Users\USER\AppData\Local\Temp\ipykernel_14680\3197859161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moviesnew["belongs_to_collection"]=moviesnew["belongs_to_collection"].apply(lambda x:[i.replace(" ","") for i in x])
C:\Users\USER\AppData\Local\Temp\ipykernel_14680\3197859161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moviesnew["genres"]=moviesnew["genres"].apply(lambda x:[i.replace(" ","") for i in x])
C:\Users\USER\AppData\Local\Temp\ipykernel_14680\3197859161.py:3: SettingWithCopyWar

In [15]:
moviesnew["tag"]=moviesnew["genres"]+moviesnew["belongs_to_collection"]+moviesnew["overview"]+moviesnew["title"]+moviesnew["tagline"]+moviesnew["crew"]+moviesnew["cast"]

C:\Users\USER\AppData\Local\Temp\ipykernel_14680\3611071775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moviesnew["tag"]=moviesnew["genres"]+moviesnew["belongs_to_collection"]+moviesnew["overview"]+moviesnew["title"]+moviesnew["tagline"]+moviesnew["crew"]+moviesnew["cast"]


In [16]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [17]:
moviesnew['tag']=moviesnew['tag'].apply(lambda x:" ".join(x))
moviesnew.to_csv("moviesnew.csv",index=False)
new_ds=new_ds=moviesnew[["id","tag",'title1']]
new_ds.to_csv("new_ds.csv",index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_14680\1705346979.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moviesnew['tag']=moviesnew['tag'].apply(lambda x:" ".join(x))


In [18]:
moviesnew['tag']=moviesnew['tag'].apply(stem)

C:\Users\USER\AppData\Local\Temp\ipykernel_14680\2569625649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moviesnew['tag']=moviesnew['tag'].apply(stem)


In [19]:
moviesnew['tag']=moviesnew['tag'].apply(lambda x:x.lower())

C:\Users\USER\AppData\Local\Temp\ipykernel_14680\397672902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moviesnew['tag']=moviesnew['tag'].apply(lambda x:x.lower())


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=6000,stop_words="english")
vectors=cv.fit_transform(moviesnew["tag"]).toarray()


In [21]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vectors)


In [22]:
np.save('similaritybwmovies.npy',similarity)

In [23]:
from surprise import Reader,Dataset,SVD
from surprise.model_selection import cross_validate
reader=Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [24]:
from surprise.model_selection import KFold
kf = KFold(n_splits=2000)
kf.split(data)


<generator object KFold.split at 0x000001DC95E869D0>

In [25]:
del similarity

In [26]:
svd=SVD()
set1=data.build_full_trainset()
svd.fit(set1)


In [27]:
import joblib
joblib.dump(svd,'../models/svd_model.pkl')

['../models/svd_model.pkl']